### My Analysis after reviewing the results
* District summary indicates an overall pass of 65% of students with higher pass percent in reading compared to math
* Charter schools have relatively close averages in math and reading pass percent compared to district schools
* Top schools with overall pass percentage comes from Charter schools 
* Per student spending reflects inversely on performance (higher the spending, lower overall passing)
  Optimal spend will be the recommendation.
* Comparing averages based on school size, small and medium are relatively close as size difference is not too great. 
  Larger the size, larger is reduction in performance. 
* Looking at scores by grade, all schools maintain somewhat same average (not too much of deviation) from 9th thru 12th. 
  So, we can conclude that focus on education for kids be done early on in life.

In [1]:
# Dependencies
import pandas as pd

In [2]:
# Set file(s) to read 
school_data_csvfile = "Resources/schools_complete.csv"
student_data_csvfile = "Resources/students_complete.csv"

In [3]:
# Read School and Student csv files and store into Pandas DataFrames
school_data = pd.read_csv(school_data_csvfile)
student_data = pd.read_csv(student_data_csvfile)

In [4]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [5]:
#Get totals from the school/student dataset
total_students = len(student_data["Student ID"].unique())
total_schools = len(school_data["School ID"].unique())
total_budget = school_data["budget"].sum()

#Calculate totals from merged dataset
avg_math_score = school_data_complete["math_score"].mean()
avg_reading_score = school_data_complete["reading_score"].mean()

#percentage of students with passing math score (70 or above)
pass_condition_math = (school_data_complete["math_score"] >= 70)
total_passing_math = len(school_data_complete[pass_condition_math])

#percentage of students with passing reading score (70 or above)
pass_condition_reading = (school_data_complete["reading_score"] >= 70)
total_passing_reading = len(school_data_complete[pass_condition_reading])

#Overall pass (both math and reading)
total_overall_pass = len(school_data_complete[pass_condition_math & pass_condition_reading])

#Percentage values for math, reading and overall
math_pass_pct = (total_passing_math/total_students) * 100
reading_pass_pct = (total_passing_reading/total_students) * 100
overall_pass_pct = (total_overall_pass/total_students) * 100


In [6]:
#Create dataframe to hold District totals
district_totals = pd.DataFrame({"Total Schools": [total_schools], "Total Students": [total_students],
                          "Total Budget": [total_budget], "Average Math Score": [avg_math_score],
                          "Average Reading Score": [avg_reading_score], "% Passing Math": [math_pass_pct],
                          "% Passing Reading": [reading_pass_pct], "% Overall Passing": [overall_pass_pct]
                         })

# format total student and total budget columns 
district_totals["Total Students"] = district_totals["Total Students"].astype(float).map("{:,.0f}".format)
district_totals["Total Budget"] = district_totals["Total Budget"].astype(float).map("${:,.2f}".format)


### District Summary

In [7]:
# Display District Summary
district_totals

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [8]:
#Create dataframe to hold school totals
grouped_school = school_data_complete.groupby(["school_name"]).agg({
    'type': "first", 
    'Student ID': pd.Series.count,
    'budget': "first",
    'math_score': pd.Series.mean,
    'reading_score': pd.Series.mean
    })

grouped_school["Per Student Budget"] = grouped_school["budget"]/grouped_school["Student ID"]

In [9]:
# Get filtered dataframe for passing math score, reading and overall (math and reading)
reduced_df = school_data_complete[["school_name", "Student ID", "grade", "reading_score", "math_score"]]
passed_math = reduced_df.loc[reduced_df["math_score"] >= 70]
passed_reading = reduced_df.loc[reduced_df["reading_score"] >= 70]
passed_overall = reduced_df.loc[(reduced_df["math_score"] >= 70) & (reduced_df["reading_score"] >= 70)]

#Groupby school name with aggregation and renamed column for result (result = count of students who passed math)   
math_df = passed_math.groupby(["school_name"]).agg(math_pass_count = ("Student ID","count"))
reading_df = passed_reading.groupby(["school_name"]).agg(reading_pass_count = ("Student ID","count"))
overall_df = passed_overall.groupby(["school_name"]).agg(overall_pass_count = ("Student ID","count"))
                
#add % columns to grouped_school dataframe
grouped_school["% Passing Math"] =  math_df["math_pass_count"]/grouped_school["Student ID"] * 100
grouped_school["% Passing Reading"] =  reading_df["reading_pass_count"]/grouped_school["Student ID"] * 100
grouped_school["% Overall Passing"] =  overall_df["overall_pass_count"]/grouped_school["Student ID"] * 100


In [10]:
# Remove index column label
grouped_school.index.name = None

# Rename column names
grouped_school = grouped_school.rename(columns={"type":"School Type", "Student ID":"Total Students", 
                                                "budget":"Total School Budget", "math_score":"Average Math Score",
                                                "reading_score":"Average Reading Score"})
                              
# Re-organize columns
school_totals = grouped_school[["School Type","Total Students","Total School Budget","Per Student Budget",
                               "Average Math Score","Average Reading Score","% Passing Math",
                               "% Passing Reading", "% Overall Passing"]]

school_totals["Total School Budget"] = school_totals["Total School Budget"].astype(float).map("${:,.2f}".format)
school_totals["Per Student Budget"] = school_totals["Per Student Budget"].astype(float).map("${:,.2f}".format)


### School Summary

In [11]:
# Display School Summary
school_totals

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


### Top five schools based on % Overall Passing

In [12]:
# Sort and display the top five performing schools by % overall passing.
school_totals = school_totals.sort_values("% Overall Passing", ascending=False)
school_totals.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


### Bottom five schools based on % Overall Passing

In [13]:
# Sort and display the five worst-performing schools by % overall passing.
school_totals = school_totals.sort_values("% Overall Passing", ascending=True)
school_totals.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [14]:
# Get list of math score averages per grade (9 thru 12) grouped by School

# create new dataframe with just the required columns
math_df = school_data_complete[["school_name", "grade", "math_score"]]
grouped_math = math_df.loc[math_df["grade"] == "9th",["school_name","math_score"]].groupby("school_name")
# Save grade 9 math average from group as a series
mathavg_grade9 = grouped_math["math_score"].mean()

# Repeat for grades 10, 11 and 12  
grouped_math = math_df.loc[math_df["grade"] == "10th",["school_name","math_score"]].groupby("school_name")
mathavg_grade10 = grouped_math["math_score"].mean()

grouped_math = math_df.loc[math_df["grade"] == "11th",["school_name","math_score"]].groupby("school_name")
mathavg_grade11 = grouped_math["math_score"].mean()

grouped_math = math_df.loc[math_df["grade"] == "12th",["school_name","math_score"]].groupby("school_name")
mathavg_grade12 = grouped_math["math_score"].mean()

# Create new Dataframe for math scores by Grade
math_avg_scores = pd.DataFrame({"9th": mathavg_grade9, "10th": mathavg_grade10,
                            "11th": mathavg_grade11, "12th": mathavg_grade12})


### Math Scores By Grade

In [15]:
# Remove index title
math_avg_scores.index.name = None
math_avg_scores

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


In [16]:
# Get list of reading score averages per grade (9 thru 12) grouped by School

# create new dataframe with just the required columns
reading_df = school_data_complete[["school_name", "grade", "reading_score"]]
grouped_reading = reading_df.loc[reading_df["grade"] == "9th",["school_name","reading_score"]].groupby("school_name")
# Save grade 9 math average from group as a series
readingavg_grade9 = grouped_reading["reading_score"].mean()

# Repeat for grades 10, 11 and 12  
grouped_reading = reading_df.loc[reading_df["grade"] == "10th",["school_name","reading_score"]].groupby("school_name")
readingavg_grade10 = grouped_reading["reading_score"].mean()

grouped_reading = reading_df.loc[reading_df["grade"] == "11th",["school_name","reading_score"]].groupby("school_name")
readingavg_grade11 = grouped_reading["reading_score"].mean()

grouped_reading = reading_df.loc[reading_df["grade"] == "12th",["school_name","reading_score"]].groupby("school_name")
readingavg_grade12 = grouped_reading["reading_score"].mean()

# Create new Dataframe for math scores by Grade
reading_avg_scores = pd.DataFrame({"9th": readingavg_grade9, "10th": readingavg_grade10,
                            "11th": readingavg_grade11, "12th": readingavg_grade12})


### Reading Scores By Grade

In [17]:
# Remove index title
reading_avg_scores.index.name = None
reading_avg_scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


In [18]:
# School performances based on average Spending Ranges (Per Student) 

# Create dataframe for student performance from school_totals with just the required columns
sp = school_totals.iloc[:, [3,4,5,6,7,8]].copy()

# Remove formatting for student budget colunn and convert data type to int 
# Note: default of regex=True, so set to false for the replace to avoid warnings
sp["Per Student Budget"] = sp["Per Student Budget"].str.replace("$", "", regex=False)
sp["Per Student Budget"] = sp["Per Student Budget"].astype(float)
sp["Per Student Budget"] = sp["Per Student Budget"].astype(int)

# Using below qcut (equal partitions) got the average values as 583, 628, 639 and 655
# sp["Spending Range"] = pd.qcut(sp["Per Student Budget"], 4)
# sp = sp.groupby("Spending Range")
# sp.max()

# Rounding edges to match expected results, using ranges 0-585, 585-630, 630-645, 645-680
# Create custom bins and group labels
bins = [0, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

# Get results from cut() function
sp["Spending Ranges (Per Student)"] = pd.cut(sp["Per Student Budget"], bins=bins, 
                                             labels=group_names, include_lowest=False)

# Note: results from displaying the dataframe gives the following 
# <585 has 4 counts; 585-630 has 4 school counts; 630-645 has 4 schools and 645-680 has 3 schools

# Drop student budget column (to match expected results)
sp = sp.drop("Per Student Budget", axis=1)

# Group by range and find average
sp = sp.groupby("Spending Ranges (Per Student)")


### Scores By School Spending

In [19]:
# set option to round float to 2 decimals  
pd.set_option("display.float_format", '{:,.2f}'.format)

# Display results - average of the group
sp.mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


In [20]:
# School performances based on School size 

# Reset float format option (results going forward require display of all decimals)
pd.reset_option("display.float_format")

# Create dataframe for school size performance from school_totals with just the required columns
performance_by_size = school_totals.iloc[:, [1,4,5,6,7,8]].copy()

performance_by_size["Total Students"] = performance_by_size["Total Students"].astype(int)

# Add school size series - information got from school table
#temp_df = school_data.copy()
#performance_by_size = temp_df.sort_values("school_name") 

# Change index to school_name
#indexed_df = sorted_df.set_index("school_name")

# Get size column as list and add to per_size dataframe
#size_col = indexed_df["size"].tolist()
#performance_by_size["size"] = size_col

# Create custom bins and group labels
Bins = [0, 1000, 2000, 5000]
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Get results from cut() function
performance_by_size["School Size"] = pd.cut(performance_by_size["Total Students"], 
                                            bins=Bins, labels=size_names, include_lowest=False)

# Drop size column (to match expected results)
performance_by_size = performance_by_size.drop("Total Students", axis=1)


# Group by range
performance_by_size = performance_by_size.groupby("School Size")

### Scores By School Size

In [21]:
# Display results - average of the group
performance_by_size.mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [22]:
# School performances based on School type 

# Create dataframe for school size performance from school_totals with just the required columns
performance_by_type = school_totals.iloc[:, [0,4,5,6,7,8]].copy()

# Drop index (to match expected results)
performance_by_type = performance_by_type.reset_index(drop=True)
score_by_type = performance_by_type.groupby("School Type").mean()

### Scores By School Type

In [23]:
# Display results - average of the group
score_by_type


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
